In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f

# **Table of Contents**

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Dataset description](#1)
2. [Configuration of Spark](#2)
3. [Data loading](#3)
4. [Data preprocessing](#4)
5. [Transformation Stage](#5)
    - [Get the general information about dataset. (Such as Schema, Columns,
number of columns and rows etc.) Describe the dataset using this
information](#5.1)
    - [Get statistics about numerical columns. Describe the dataset using this
information. How could this be useful for further analysis?](#5.2)
    - [Come up with your own 6 or more business questions to the data](#5.3)
    
</div>

# **1. Dataset description** <a id='1'></a>

<details><summary>Click here for the dataset description</summary>

## **Dataset Description (IMDb dataset)**

### **Files**
* **name.basics.tsv** - 
* **title.akas.tsv** - 
* **title.basics.tsv** - 
* **title.crew.tsv** - 
* **title.episode.tsv** - 
* **title.principals.tsv** - 
* **title.ratings.tsv** - 

### **name.basics.tsv**

|      Name         |    Type          |   Description                                     |
| -------------     | ---------------- | ------------------------------------------------- |
|     nconst        |   string         | alphanumeric unique identifier of the name/person |
|   primaryName     | string           | name by which the person is most often credited   |
|     birthYear     | in YYYY format   | birth year                                        |
|     deathYear     | in YYYY format   | if applicable, else '\N'                          |
| primaryProfession | array of strings | the top-3 professions of the person               |
|   knownForTitles  | array of tconsts | titles the person is known for                    |

### **title.akas.tsv**

|      Name       |    Type   |   Description    |
| -------------   | --------- | ---------------- |
| titleId         | string    | a tconst, an alphanumeric unique identifier of the title |
| ordering        | integer   | a number to uniquely identify rows for a given titleId |
| title           | string    | the localized title |
| region          | string    | the region for this version of the title |
| language        | string    | the language of the title |
| types           | array     | Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning |
| attributes      | array     | Additional terms to describe this alternative title, not enumerated |
| isOriginalTitle | boolean   | 0: not original title; 1: original title |

### **title.basics.tsv**

|      Name      |    Type      |   Description   |
| -------------  | ------------ | ----------------|
| tconst         | string       | alphanumeric unique identifier of the title |
| titleType      | string       | the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc) |
| primaryTitle   | string       | the more popular title / the title used by the filmmakers on promotional materials at the point of release |
| originalTitle  | string       | original title, in the original language |
| isAdult        | boolean      | 0: non-adult title; 1: adult title |
| startYear      | YYYY         | represents the release year of a title. In the case of TV Series, it is the series start year |
| endYear        | YYYY         | TV Series end year. ‘\N’ for all other title types |
| runtimeMinutes | integer      | primary runtime of the title, in minutes |
| genres         | string array | includes up to three genres associated with the title |

### **title.crew.tsv**

|      Name     |    Type          |   Description                               |
| ------------- | ---------------  | ------------------------------------------- |
| tconst        | string           | alphanumeric unique identifier of the title |
| directors     | array of nconsts | director(s) of the given title              |
| writers       | array of nconsts | writer(s) of the given title                |

### **title.episode.tsv**

|      Name     |    Type   |   Description                                   |
| ------------- | --------- | ----------------------------------------------- |
| tconst        | string    | alphanumeric identifier of episode              |
| parentTconst  | string    | alphanumeric identifier of the parent TV Series |
| seasonNumber  | integer   | season number the episode belongs to            |
| episodeNumber | integer   | episode number of the tconst in the TV series   |

### **title.principals.tsv**

|      Name     |    Type   |   Description                                             |
| ------------- | --------- | --------------------------------------------------------- |
| tconst        | string    | alphanumeric unique identifier of the title               |
| ordering      | integer   | a number to uniquely identify rows for a given titleId    |
| nconst        | string    | alphanumeric unique identifier of the name/person         |
| category      | string    | the category of job that person was in                    |
| job           | string    | the specific job title if applicable, else '\N'           |
| characters    | string    | the name of the character played if applicable, else '\N' |

### **title.ratings.tsv**

|      Name     |    Type   |   Description                                        |
| ------------- | --------- | ---------------------------------------------------- |
| tconst        | string    | alphanumeric unique identifier of the title          |
| averageRating | float     | weighted average of all the individual user ratings  |
| numVotes      |  integer  | number of votes the title has received               |
    
</details>

# **2. Configuration of Spark** <a id='2'></a>

In [2]:
path = 'C:/files/grid-dynamics'

In [3]:
spark_session = (
    SparkSession.builder
        .master('local')
        .appName('project')
        .config(conf=SparkConf())
        .getOrCreate()
)

spark_session.conf.set('spark.sql.repl.eagerEval.enabled', True)

Defining the schemas

In [4]:
name_basics_schema = t.StructType([
    t.StructField('nconst', t.StringType(), True),
    t.StructField('primaryName', t.StringType(), True),
    t.StructField('birthYear', t.DateType(), True),
    t.StructField('deathYear', t.DateType(), True),
    t.StructField('primaryProfession', t.StringType(), True),
    t.StructField('knownForTitles', t.StringType(), True)
])

akas_schema = t.StructType([
    t.StructField('titleId', t.StringType(), True),
    t.StructField('ordering', t.IntegerType(), True),
    t.StructField('title', t.StringType(), True),
    t.StructField('region', t.StringType(), True),
    t.StructField('language', t.StringType(), True),
    t.StructField('types', t.StringType(), True),
    t.StructField('attributes', t.StringType(), True),
    t.StructField('isOriginalTitle', t.BooleanType(), True)
])

title_basics_schema = t.StructType([
    t.StructField('tconst', t.StringType(), True),
    t.StructField('titleType', t.StringType(), True),
    t.StructField('primaryTitle', t.StringType(), True),
    t.StructField('originalTitle', t.StringType(), True),
    t.StructField('isAdult', t.BooleanType(), True),
    t.StructField('startYear', t.DateType(), True),
    t.StructField('endYear', t.DateType(), True),
    t.StructField('runtimeMinutes', t.IntegerType(), True),
    t.StructField('genres', t.StringType(), True),
])

crew_schema = t.StructType([
    t.StructField('tconst', t.StringType(), True),
    t.StructField('directors', t.StringType(), True),
    t.StructField('writers', t.StringType(), True),
])

episode_schema = t.StructType([
    t.StructField('tconst', t.StringType(), True),
    t.StructField('parentTconst', t.StringType(), True),
    t.StructField('seasonNumber', t.IntegerType(), True),
    t.StructField('episodeNumber', t.IntegerType(), True),
])

principals_schema = t.StructType([
    t.StructField('tconst', t.StringType(), True),
    t.StructField('ordering', t.IntegerType(), True),
    t.StructField('nconst', t.StringType(), True),
    t.StructField('category', t.StringType(), True),
    t.StructField('job', t.StringType(), True),
    t.StructField('characters', t.StringType(), True)
])

ratings_schema = t.StructType([
    t.StructField('tconst', t.StringType(), True),
    t.StructField('averageRating', t.FloatType(), True),
    t.StructField('numVotes', t.IntegerType(), True)
])

# **3. Data loading** <a id='3'></a>

Reading the dataset

In [5]:
name_basics = spark_session.read.csv(f'{path}/name.basics.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=name_basics_schema,
                                    dateFormat='yyyy')

akas = spark_session.read.csv(f'{path}/title.akas.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=akas_schema)

title_basics = spark_session.read.csv(f'{path}/title.basics.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=title_basics_schema,
                                    dateFormat='yyyy')

crew = spark_session.read.csv(f'{path}/title.crew.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=crew_schema)

episode = spark_session.read.csv(f'{path}/title.episode.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=episode_schema)

principals = spark_session.read.csv(f'{path}/title.principals.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=principals_schema)

ratings = spark_session.read.csv(f'{path}/title.ratings.tsv', 
                                    sep=r'\t', 
                                    header=True,
                                    nullValue='null',
                                    schema=ratings_schema)

# **4. Data preprocessing** <a id='4'></a>

Replacing '\N' with null value

In [6]:
name_basics = name_basics.withColumn('primaryProfession', f.when(f.col('primaryProfession') == r'\N', None).otherwise(f.col('primaryProfession')))
name_basics = name_basics.withColumn('knownForTitles', f.when(f.col('knownForTitles') == r'\N', None).otherwise(f.col('knownForTitles')))

title_basics = title_basics.withColumn('genres', f.when(f.col('genres') == r'\N', None).otherwise(f.col('genres')))

crew = crew.withColumn('writers', f.when(f.col('writers') == r'\N', None).otherwise(f.col('writers')))
crew = crew.withColumn('directors', f.when(f.col('directors') == r'\N', None).otherwise(f.col('directors')))

principals = principals.withColumn('job', f.when(f.col('job') == r'\N', None).otherwise(f.col('job')))
principals = principals.withColumn('characters', f.when(f.col('characters') == r'\N', None).otherwise(f.col('characters')))

akas = akas.withColumn('region', f.when(f.col('region') == r'\N', None).otherwise(f.col('region')))
akas = akas.withColumn('language', f.when(f.col('language') == r'\N', None).otherwise(f.col('language')))
akas = akas.withColumn('types', f.when(f.col('types') == r'\N', None).otherwise(f.col('types')))
akas = akas.withColumn('attributes', f.when(f.col('attributes') == r'\N', None).otherwise(f.col('attributes')))



Converting strings to array of strings

In [7]:
name_basics = name_basics.withColumn('primaryProfession', f.split('primaryProfession', ','))
name_basics = name_basics.withColumn('knownForTitles', f.split('knownForTitles', ','))

title_basics = title_basics.withColumn('genres', f.split('genres', ','))

crew = crew.withColumn('directors', f.split('directors', ','))
crew = crew.withColumn('writers', f.split('writers', ','))

Removing unnecessary symbols

In [8]:
principals = principals.withColumn('characters', f.regexp_replace('characters', r'[\[\]"]', ''))

# **5. Transformation Stage** <a id='5'></a>

## **1. Get the general information about dataset. (Such as Schema, Columns, number of columns and rows etc.) Describe the dataset using this information** <a id='5.1'></a>

In [10]:
datasets = [(name_basics, 'name_basics'), (akas, 'akas'), (title_basics, 'title_basics'), (crew, 'crew'), 
            (episode, 'episode'), (principals, 'principals'), (ratings, 'ratings')]

In [11]:
print('Schemas\n')

for dataset, name in datasets:
    print(f'Dataset: {name}: {dataset.printSchema()}')

Schemas

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: date (nullable = true)
 |-- deathYear: date (nullable = true)
 |-- primaryProfession: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- knownForTitles: array (nullable = true)
 |    |-- element: string (containsNull = false)

Dataset: name_basics: None
root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: boolean (nullable = true)

Dataset: akas: None
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: boolean (nullable = true)
 |-- startYear: date (nullable = true)
 |-- end

In [12]:
print('Columns\n')

for dataset, name in datasets:
    print(f'Dataset: {name}: {dataset.columns}, number of columns: {len(dataset.columns)}, number of rows: {dataset.count()}\n')

Columns

Dataset: name_basics: ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], number of columns: 6, number of rows: 12698644

Dataset: akas: ['titleId', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle'], number of columns: 8, number of rows: 36642385

Dataset: title_basics: ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'], number of columns: 9, number of rows: 10017011

Dataset: crew: ['tconst', 'directors', 'writers'], number of columns: 3, number of rows: 10019208

Dataset: episode: ['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber'], number of columns: 4, number of rows: 7620834

Dataset: principals: ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], number of columns: 6, number of rows: 57243863

Dataset: ratings: ['tconst', 'averageRating', 'numVotes'], number of columns: 3, number of rows: 1331252



## **2. Get statistics about numerical columns. Describe the dataset using this information. How could this be useful for further analysis?** <a id='5.2'></a>

In [14]:
akas.describe(['ordering'])

summary,ordering
count,36642385
mean,4.1506784015287215
stddev,3.939129056143955
min,1
max,249


In [15]:
title_basics.describe(['runtimeMinutes'])

summary,runtimeMinutes
count,2970701
mean,43.505853332260635
stddev,81.48975978468617
min,0
max,59460


In [17]:
episode.describe(['seasonNumber', 'episodeNumber'])

summary,seasonNumber,episodeNumber
count,6057523,6057523
mean,3.7390189026108525,402.8005742611295
stddev,24.441675637575482,1306.4123181308778
min,1,0
max,2021,97251


In [18]:
principals.describe(['ordering'])

summary,ordering
count,57243863
mean,4.610154489399152
stddev,2.7856930662973065
min,1
max,10


In [19]:
ratings.describe(['averageRating', 'numVotes'])

summary,averageRating,numVotes
count,1331252,1331252
mean,6.955515265382489,1038.735545937208
stddev,1.3822064108967602,17489.990386451816
min,1.0,5
max,10.0,2767112


## **3. Come up with your own 6 or more business questions to the data** <a id='5.3'></a>

In [9]:
akas.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|    null|imdbDisplay|                null|           null|
|tt0000001|       2|          Carmencita|    DE|    null|       null|       literal title|           null|
|tt0000001|       3|Carmencita - span...|    HU|    null|imdbDisplay|                null|           null|
|tt0000001|       4|          Καρμενσίτα|    GR|    null|imdbDisplay|                null|           null|
|tt0000001|       5|          Карменсита|    RU|    null|imdbDisplay|                null|           null|
|tt0000001|       6|          Carmencita|    US|    null|imdbDisplay|                null|           null|
|tt0000001|       7|          Carmenc

How many ukrainian titles in akas dataframe?

In [10]:
akas.where(akas.region == 'UA').count()

27365

What are the 5 most common regions?

In [12]:
akas.groupBy('region') \
    .count() \
    .sort(f.desc('count')) \
    .limit(5) \
    .show()

+------+-------+
|region|  count|
+------+-------+
|    DE|4393871|
|    FR|4389198|
|    JP|4388493|
|    IN|4330645|
|    ES|4308577|
+------+-------+

